In [1]:
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
from datetime import datetime, timedelta
from typing import List
from termcolor import colored

from langchain.chat_models import AzureChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

In [3]:
USER_NAME = "Person A"
LLM = AzureChatOpenAI(
    deployment_name="snsai-gpt35-0301",
    openai_api_type="azure",
    openai_api_base="https://sns-zero.openai.azure.com",
    openai_api_version="2023-03-15-preview",
    openai_api_key="c56b474e8cbc49038a765a8eaf908281",
    max_tokens=1500
)

OPENAI_API_BASE = "http://ai-ol.sns.sohu.com/v1"
OPENAI_API_KEY = "c030bd5eaa954d469aea6a3869643f23"
MODEL = "text-embedding-ada-002"

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

In [5]:
import math
import faiss


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings(openai_api_base=OPENAI_API_BASE, openai_api_key=OPENAI_API_KEY)
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [6]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=tommies_memory,
)

In [7]:
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
There are no statements provided to summarize Tommie's core characteristics.


In [9]:
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]

for observation in tommie_observations:
    tommie.memory.add_memories(observation)

In [10]:
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is a person who has fond memories of his childhood pet, Bruno, and tends to remember and reflect on those memories. He is tired from driving and in need of rest and food. He is also sensitive to his surroundings, noticing the noise level of the road and the presence of a neighbor's cat.


In [11]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [12]:
interview_agent(tommie, "What do you like to do?")

'Tommie said "I really enjoy design, it\'s always been a passion of mine. But right now, I\'m mainly focused on looking for a job. How about you, what do you like to do?"'

In [13]:
interview_agent(tommie, "What are you looking forward to doing today?")

'Tommie said "Well, today I\'m mostly looking forward to getting some rest and food after the long drive. But tomorrow I plan on starting my job hunt in earnest. What about you, what\'s on your agenda for today?"'

In [14]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Honestly, I\'m feeling pretty anxious about my job search. It\'s been a stressful process so far. But I\'m trying to stay positive and keep pushing forward. How about you, what worries you?"'

In [15]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [16]:

# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie may feel frustrated or annoyed by the noise, but may not take any action depending on his level of tiredness.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie may feel relieved or excited about getting some caffeine to help him feel more alert.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie may feel frustrated or annoyed by forgetting the coffee filters, but will likely continue to search for them in the boxes.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie may feel relieved or satisfied after finding the coffee filters and making his coffee.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie may feel disappointed or regretful about the taste of the coffee, but may still finish it to satisfy his need for caffeine.
Tommie checks his email and sees

In [17]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a mixed bag, to be honest. I went to a job fair earlier but didn\'t have much luck, and then I played frisbee and got hit in the face with it. But I also dropped off my resume at a few places and saw a sign for another job fair coming up. So there\'s some good and bad, I guess."'

In [18]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "I rely on coffee to get me through the day, but sometimes it\'s hit or miss. Today\'s cup was a miss unfortunately."'

In [19]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, my childhood dog was the best. His name was Bruno, and he was a golden retriever. We used to play together all the time, and he was always there for me when I needed him. I miss him a lot, but I have some great memories of him."'

In [20]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


eve = GenerativeAgent(
    name="Eve",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [21]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]
for observation in eve_observations:
    eve.memory.add_memory(observation)

In [22]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is helpful to her coworkers, enjoys playing tennis and maintaining friendships, is attentive to her colleagues' conversations, takes care of herself by eating breakfast, and is punctual with waking up to her alarm.


In [23]:
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling pretty good, thanks for asking. Just trying to stay on top of things and be helpful where I can. How about you?"'

In [24]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said "I\'ve overheard some negative comments about Tommie, but I don\'t personally know them well enough to have an opinion. Is there something specific you\'re curious about?"'

In [25]:
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

'Eve said "That\'s interesting. I don\'t know much about Tommie\'s job search, but I hope they find something that they enjoy. Is there anything specific you think they would be interested in or good at?"'

In [26]:
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "I understand. I\'ll try my best to keep the conversation going and ask him some questions. Is there anything specific you would like me to ask him about?"'

Dialogue between Generative Agents

In [27]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [28]:
agents = [tommie, eve]
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

Eve said "Sure, I'd be happy to share my experience with you. What specifically would you like to know?"
Tommie said "Can you tell me more about your job search experience? I'm feeling a bit discouraged with my own search."
Eve said "Of course, I'm happy to share my experience with you. When I was searching for a job, I found it helpful to network and reach out to contacts in the industry. Have you tried that approach? And what type of job are you looking for?"
Tommie said "That's a great suggestion, thanks! I haven't tried networking yet, but I'll definitely look into it. I'm mostly looking for design-related jobs. Have you heard of any openings in that field?"
Eve said "I'm not aware of any specific openings at the moment, but I can keep an eye out and let you know if I hear of anything. What type of design work are you interested in?"
Tommie said "Thanks for offering to keep an eye out for me. I'm mostly interested in graphic design and web design. Do you have any tips on how to imp

Let's interview our agents after their conversation

In [29]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is a person who values memories and has a positive association with his childhood pet, Bruno. He is tired from a long drive and tries to get some rest in his new home, which is noisy due to the road. He is also hungry and may be feeling overwhelmed with the task of exploring his new city and finding job opportunities. However, he shows motivation and hope by attending a job fair and applying for jobs. Overall, Tommie is a person who is adaptable and proactive in his pursuit of a new life.


In [30]:
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and supportive coworker who enjoys playing tennis and eating porridge. She is attentive and responsive to her colleagues' needs and is always willing to offer advice and assistance. Eve is also open-minded and non-judgmental, even when others speak negatively about people she doesn't know well. She is knowledgeable about the industry and offers useful suggestions for job seekers. Overall, Eve is a kind and knowledgeable team player who values collaboration and communication.


In [31]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "It was really helpful actually. She gave me some great advice on improving my portfolio and suggested some online platforms to showcase my work. And she even offered to keep an eye out for job openings in my field. I\'m feeling a bit more hopeful now."'

In [32]:
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "It was a productive conversation. Tommie had some great questions and I was happy to offer some advice and suggestions. How do you know Tommie?"'

In [33]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "I don\'t think there\'s anything specific I wish I would have said. I tried to offer advice and suggestions based on my own experience and knowledge of the industry. But if there\'s anything else Tommie needs help with, I\'m happy to continue the conversation and offer assistance."'